<a href="https://colab.research.google.com/github/Shahabshms/Numerical_Methods_for_ML_and_AI_Solution_4/blob/main/_4301_HW4_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3. Write a Python function that takes as input the matrices $A$ and $Q$, an initial guess for the completion, $B$, and a number of iterations and returns the the result of applying projected
gradient descent, starting at $B$, for the specified number of iterations.

The goal here is to minimize function
\begin{align*}
    g(B) = \frac{1}{2} \sum_{\substack{i,j\in\{1,\dots,n\}\\ Q_{i,j}=0}} (A_{i,j} - B_{i,j})^2
\end{align*}
such that $$B\succeq 0.$$


So, for all $i,j\in\{1,\dots,n\}$, we have
\begin{align*}
\cfrac{\partial g(B)}{\partial B_{i,j}} = \left\{
\begin{array}{l,l}
 -(A_{i,j} - B_{i,j})
&\text{if } Q_{i,j} = 0,\\
\cfrac{\partial g(B)}{\partial B_{j,i}} & \text{if } Q_{i,j} = 1 \text{and }Q_{j,i} = 0,\\
0 & \text{if } Q_{i,j} = 1 \text{and } Q_{j,i} = 1.
\end{array}\right.
\end{align*}
And in the following function, the gradient will be constructed based on these derivatives. 

In [ ]:
def get_gradient(B,A,Q):
  gradient = np.zeros([B.shape[0],B.shape[0]])
  for i,j in itertools.product(range(B.shape[0]),range(B.shape[0])):
    if Q[i,j] == 0:
      gradient[i,j] = B[i,j] - A[i,j]
      if Q[j,i] == 1:
        gradient[j,i] = gradient[i,j]
  return gradient


As always, in each iteration of the gradient descent, we need to update our variables. So
$$B^{\text{new}} = B^{\text{old}} - \eta \nabla_B g(B)$$
But we have to make sure that at each time, $B^{new}$ remains positive semidefinte. This is why we need to implement Projected Gradient Descent. The next function is responsible for getting $B^{\text{new}}$ and project it into the convex set of positive semidefinite matrices.   

In [ ]:
def projection(B):
  eig_vals, eig_vecs = np.linalg.eig(B)
  eig_vals[eig_vals < 0] = 0
  B = eig_vecs*np.diag(eig_vals)*np.transpose(eig_vecs)
  return B

And, the final function is the Projected Gradient Descent. 

In [ ]:
def projected_gradient_descent(initial_B,original_matrix,Q,max_iterations):
  B = initial_B
  A = original_matrix

  for iteration in range(max_iterations):
    nu = 2 / (2 + iteration) # step size of choice. It is better that you try different step sizes. 

    gradient = get_gradient(B,A,Q)
    B = B - nu * gradient # Update B. 
    B = projection(B)

  return B

Notice that the proble is convex. So changing the initial point must not change the final answer, as long as the initial point is positive semidefinite.

In [ ]:
import itertools
import numpy as np

A = np.matrix([[1,0,80],[1,0,0],[None,None,5]])
initial_B = np.matrix([[1,0,8],[0,5,6],[1,2,3]])
Q = (A == None) + 0
B = projected_gradient_descent(initial_B,A,Q,100)
print(A,'\n')


print(B,'\n')
print(np.linalg.eigvals(B),'\n-------')

initial_B = np.matrix([[1,0,0],[4,5,6],[1,0,9]])
B = projected_gradient_descent(initial_B,A,Q,100)
print(B,'\n')
print(np.linalg.eigvals(B))



[[1 0 80]
 [1 0 0]
 [None None 5]] 

[[4.04717961e+01 4.87474668e-01 4.14962358e+01]
 [4.87474668e-01 5.88319468e-03 4.99813839e-01]
 [4.14962358e+01 4.99813839e-01 4.25466066e+01]] 

[ 8.30242742e+01 -4.18592163e-15  1.16592410e-05] 
-------
[[4.04721920e+01 4.87470003e-01 4.14966425e+01]
 [4.87470003e-01 5.88593821e-03 4.99809065e-01]
 [4.14966425e+01 4.99809065e-01 4.25470243e+01]] 

[8.30250876e+01 1.42516004e-15 1.45723674e-05]
